In [1]:
!python -m venv myenv


In [ ]:
!pip install -r requirements.txt

In [ ]:
!aws s3 ls commoncrawl/

In [1]:
import duckdb
import pandas as pd

In [ ]:
--!aws s3 cp s3://crawler-works/cc_index.duckdb .

In [2]:
DB_PREFIX = './data'
DB_NAME_FULL= f'{DB_PREFIX}/cc_index.duckdb' #full
DB_NAME_15_PCT = f'{DB_PREFIX}/cc_index_15pc_sample.duckdb' #15%
DB_NAME_MICRO = f'{DB_PREFIX}/cc_index_p1pct_sample.duckdb' #0.1%
DB_NAME_NANO = f'{DB_PREFIX}/cc_index_p1pct_sample.duckdb'

DB_NAME = DB_NAME_NANO

In [3]:
con = duckdb.connect(DB_NAME)
con.install_extension("aws")
con.load_extension("aws")

con.install_extension("s3")
con.load_extension("s3")

con.install_extension("httpfs")
con.load_extension("httpfs")

In [4]:
CRAWL_SET = "CC-MAIN-2024-42"
CC_URL_S3_BASE="s3://"
CC_URL_HTTP_BASE="https://data.commoncrawl.org/"
CC_INDEX_URL = f"{CC_URL_HTTP_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/*.parquet"
CC_INDEX_S3_URL = f"{CC_URL_S3_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/*.parquet"
CC_INDEX_S3_BASE_URL = f"{CC_URL_S3_BASE}commoncrawl/cc-index/table/cc-main/warc/crawl={CRAWL_SET}/subset=warc/"
CC_INDEX_URL

'https://data.commoncrawl.org/commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/*.parquet'

In [7]:
CC_INDEX_S3_URL

's3://commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/*.parquet'

In [8]:
print(f"{CC_INDEX_S3_BASE_URL}")

s3://commoncrawl/cc-index/table/cc-main/warc/crawl=CC-MAIN-2024-42/subset=warc/


In [5]:
# Configure S3 settings for DuckDB
con.execute("""
    CALL load_aws_credentials();
""")

In [6]:
def sql(query):
  return con.execute(query).df()

In [11]:
sql('select count(*) from sample_db_nano.cc_index')

,count_star()
0,0


In [6]:
con.execute("""
  DROP SECRET IF EXISTS secret3;
  CREATE SECRET secret3 (
    TYPE S3,
    PROVIDER CREDENTIAL_CHAIN,
    CHAIN 'sso'
  );
            """)

--con.execute(f"""
  CREATE TABLE IF NOT EXISTS t1 as 
  SELECT *
    FROM read_parquet('{CC_INDEX_S3_URL}');
  DESCRIBE t1;
""").df()

In [ ]:
!ls

In [ ]:
con.execute('select count(*) from t1').df()

In [ ]:
con.execute('describe t1').df()

In [ ]:
con.execute('select * from t1 limit 3').df().transpose() 

In [ ]:
con.execute('select count(distinct  url_host_name) from t1').df()

In [ ]:
#pages per host
con.execute('select count(*)/count(distinct url_host_name) from t1').df()

In [ ]:
con.execute('select count(distinct warc_filename) from t1').df()

In [12]:
p1pct_db_path = './data/cc_index_p1pct_sample.duckdb'
p1pct_sample_fraction = ".1%"
con.execute(f"ATTACH '{p1pct_db_path}' AS sample_db_p1pct")

BinderException: Binder Error: Unique file handle conflict: Database "sample_db_p1pct" is already attached with path "./data/cc_index_p1pct_sample.duckdb", 

In [20]:
nano_db_path = './data/cc_index_nano.duckdb'
nano_sample_fraction = ".2%"
con.execute(f"ATTACH '{nano_db_path}' AS sample_db_nano")

BinderException: Binder Error: Unique file handle conflict: Database "sample_db_nano" is already attached with path "./data/cc_index_nano.duckdb", 

In [21]:
con.execute(f"""
    DROP table sample_db_nano.cc_index;
    CREATE TABLE sample_db_nano.cc_index AS
    SELECT * FROM sample_db_p1pct.cc_index_sample USING SAMPLE {nano_sample_fraction};
""")

In [22]:
sql('select count(*) from sample_db_nano.cc_index'), \
  sql('select count(*) from sample_db_p1pct.cc_index_sample')

(   count_star()
 0          4096,
    count_star()
 0       2464920)

In [28]:
files = [f for f in sql('select warc_filename from sample_db_nano.cc_index')['warc_filename']]
sample_file = files[0]

In [29]:
!wget -P data/ {CC_URL_HTTP_BASE}{sample_file}

--2024-12-17 23:12:14--  https://data.commoncrawl.org/crawl-data/CC-MAIN-2024-42/segments/1727944255165.86/warc/CC-MAIN-20241013061417-20241013091417-00064.warc.gz
Resolving data.commoncrawl.org (data.commoncrawl.org)... 2600:9000:2177:f600:1d:ef90:7bc0:93a1, 2600:9000:2177:b800:1d:ef90:7bc0:93a1, 2600:9000:2177:7800:1d:ef90:7bc0:93a1, ...
Connecting to data.commoncrawl.org (data.commoncrawl.org)|2600:9000:2177:f600:1d:ef90:7bc0:93a1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 936652090 (893M) [application/octet-stream]
Saving to: ‘data/CC-MAIN-20241013061417-20241013091417-00064.warc.gz’

CC-MAIN-20241013061 100%[===================>] 893.26M  45.3MB/s    in 25s     

2024-12-17 23:12:39 (36.2 MB/s) - ‘data/CC-MAIN-20241013061417-20241013091417-00064.warc.gz’ saved [936652090/936652090]



In [ ]:
con.execute(f"""
    CREATE TABLE sample_db_p1pct.cc_index_sample AS
    SELECT * FROM t1 USING SAMPLE {p1pct_sample_fraction};
""")

In [ ]:
!aws s3 cp cc_index_15pct_sample.duckdb s3://crawler-works/

In [ ]:
second_db_path = 'cc_index_15pct_sample.duckdb'
con.execute(f"ATTACH '{second_db_path}' AS sample_db")

In [30]:
sample_fraction = "15%"
con.execute(f"""
    CREATE TABLE sample_db.cc_index_sample AS
    SELECT * FROM t1 USING SAMPLE {sample_fraction};
""")

In [ ]:
con.execute(f"""
    select count(*) from sample_db.cc_index_sample
""").df()

In [ ]:
con.execute(f"""
    select count(*) from sample_db_p1pct.cc_index_sample
""").df()

In [ ]:
!aws s3 cp {p1pct_db_path} s3://crawler-works/

# IGNORE EVERYTHING BELOW THIS LINE

In [ ]:
!duckdb -c "\
  create table meta as SELECT * FROM '../cc/cdx-00000.jsonl'; \
  create view status_vw as Select status, count(status) as cnt from meta group by status order by cnt desc; \
  select * from meta; \
  select * from status_vw; \
" {DB_FILE}

In [ ]:
!duckdb -c ".databases" {DB_FILE}
#!duckdb -c ".open hello.db"

In [ ]:
!duckdb -c " \
create table metastats as select trim(status) as tstatus, count(trim(status)) as cstatus, round(count(*) * 100.0/sum(count(*)) over(), 2) as pct from meta group by tstatus order by cstatus desc; \
select * from metastats; \
" {DB_FILE}

In [ ]:
!duckdb -c " \
create table metastats_norobotstxt as select trim(status) as tstatus, count(trim(status)) as cstatus, round(count(*) * 100.0/sum(count(*)) over(), 2) as pct from meta where url not like '%robots.txt' group by tstatus order by cstatus desc; \
select * from metastats_norobotstxt; \
" {DB_FILE}

In [ ]:
#!duckdb -c "drop table metastats" {DB_FILE}

In [ ]:
first_line = ""
with open("../cc/cdx-00000", "r") as file:
    first_line = file.readline()

print(first_line)

In [ ]:
!echo "{first_line}" | cut -d' ' -f3- 

In [ ]:
import json
s = '{"url": "http://1.0.0.0/robots.txt", "mime": "text/html", "mime-detected": "text/html", "status": "403", "digest": "UNFBLUUDKY6EPX25BG4X7TI6J3IS2BBZ", "length": "2706", "offset": "890", "filename": "crawl-data/CC-MAIN-2024-42/segments/1727944253858.36/robotstxt/CC-MAIN-20241011230131-20241012020131-00456.warc.gz"}'
json.loads(s)